[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/MeteoSwiss/opendata-nwp-demos/blob/main/07_interpolate_to_target_altitude.ipynb)

# Interpolating ICON-CH2-EPS Temperature Forecasts

This notebook demonstrates the full workflow for accessing ICON-CH2-EPS temperature forecasts and performing vertical interpolation from model levels to a target altitude. The data is provided by MeteoSwiss as part of Switzerland’s [Open Government Data (OGD) initiative](https://www.meteoswiss.admin.ch/services-and-publications/service/open-data.html).

The core functionality is powered by the [meteodata-lab](https://meteoswiss.github.io/meteodata-lab/) library — developed in-house to simplify working with numerical weather model data. It includes the [ogd_api](https://meteoswiss.github.io/meteodata-lab/generated/meteodatalab.ogd_api.html) module for data access, along with utilities for applying temporal and spatial operators to model output, such as horizontal or vertical interpolation.

The ICON-CH1/2-EPS data is typically provided on terrain-following vertical model levels, which do not correspond to height above sea. However, for consistent comparison and analysis, it is often necessary to interpolate forecast data to these specific target altitudes. The [meteodata-lab](https://meteoswiss.github.io/meteodata-lab/) library includes the function [interpolate_k2any()](https://meteoswiss.github.io/meteodata-lab/generated/meteodatalab.operators.vertical_interpolation.html#meteodatalab.operators.vertical_interpolation.interpolate_k2any), which interpolate a field from model levels to a target altitude.

---

## 🔍 **What You’ll Do in This Notebook**

 🛰️  **Retrieve**  
    Fetch deterministic ICON-CH2-EPS forecast data (here temperature (`T`) and pressure (`P`)) using meteodata-lab's [ogd_api](https://meteoswiss.github.io/meteodata-lab/generated/meteodatalab.ogd_api.html) module.

 📉  **Interpolate to target altitude**  
    Interpolate ICON-CH2-EPS forecast data to a specific target altitude using meteodata-lab's [vertical_interpolation](https://meteoswiss.github.io/meteodata-lab/generated/meteodatalab.operators.vertical_interpolation.html#) module.

---

## Retrieving Forecasts
In this first part, we retrieve deterministic ICON-CH2-EPS temperature forecast data. To access this data, we use the [ogd_api](https://meteoswiss.github.io/meteodata-lab/generated/meteodatalab.ogd_api.html) module from the [meteodata-lab](https://meteoswiss.github.io/meteodata-lab/) library — a convenient interface for accessing numerical weather forecasts via the [STAC (SpatioTemporal Asset Catalog) API](https://data.geo.admin.ch/api/stac/static/spec/v1/apitransactional.html#tag/Data/operation/getAsset), which provides structured access to Switzerland’s open geospatial data.

### 📁  Browsing the STAC Catalog (Optional)

If you'd like to explore the ICON-CH1/2-EPS forecast datasets interactively before writing code, you can browse them directly in the STAC catalog:

&nbsp;&nbsp;&nbsp;&nbsp;🔗  [Browse the ICON-CH1-EPS collection](https://data.geo.admin.ch/browser/#/collections/ch.meteoschweiz.ogd-forecasting-icon-ch1?.language=en)

&nbsp;&nbsp;&nbsp;&nbsp;🔗  [Browse the ICON-CH2-EPS collection](https://data.geo.admin.ch/browser/#/collections/ch.meteoschweiz.ogd-forecasting-icon-ch2?.language=en)


Below is a screenshot of the ICON-CH2-EPS collection as seen in the STAC browser interface.


![browser-ch2.png](./images/browser-ch2.png)

⚙️ **Google Colab Setup**  
This cell installs all required dependencies if you're running the notebook in **Google Colab**.  
It is automatically skipped when running in a local Jupyter environment.

In [ ]:
# 📦 Google Colab Setup (skipped if not running in Colab)
import sys

def is_colab():
    return "google.colab" in sys.modules

if is_colab():
    !git clone https://github.com/MeteoSwiss/opendata-nwp-demos.git
    %cd opendata-nwp-demos
    !pip install poetry && poetry config virtualenvs.in-project true && poetry install --no-ansi
    import sys, os, pathlib
    venv = pathlib.Path(".venv")
    site = venv / "lib" / f"python{sys.version_info.major}.{sys.version_info.minor}" / "site-packages"
    sys.path.insert(0, str(site))
    os.environ["ECCODES_DEFINITION_PATH"] = str((venv / "share/eccodes-cosmo-resources/definitions").resolve())

### Creating Requests
To retrieve the forecast data, we first define an API request using the [ogd_api.Request](https://meteoswiss.github.io/meteodata-lab/generated/meteodatalab.ogd_api.html#meteodatalab.ogd_api.Request) class. In this example, we create a request to retrieve the temperature field (`T`), which we aim to interpolate.

>⏰ **Forecast Availability**: Forecast data will typically be available a couple of hours after the reference time — due to the model runtime and subsequent upload time. The data remains accessible for 24 hours after upload.

In [ ]:
from datetime import datetime, timezone
from meteodatalab import ogd_api

# Set default date of today midnight in UTC
today_midnight_utc = datetime.now(timezone.utc).replace(hour=0, minute=0, second=0, microsecond=0)


req = ogd_api.Request(
    collection="ogd-forecasting-icon-ch2",
    variable="T",
    reference_datetime=today_midnight_utc,
    perturbed=False,
    horizon=f"P0DT0H",
)

Each argument in the request serves the following purpose:

| Argument             | Description |
|----------------------|-------------|
| `collection`         | Forecast collection to use (e.g., `ogd-forecasting-icon-ch2` for ICON-CH2-EPS). |
| `variable`           | Meteorological variable of interest (`T` = temperature). |
| `reference_datetime` | Initialization time of the forecast in **UTC**, provided as either:<br>- [datetime.datetime](https://docs.python.org/3/library/datetime.html#datetime-objects) object (e.g.,<br> &nbsp; `datetime.datetime(2025, 5, 22, 9, 0, 0, tzinfo=datetime.timezone.utc)`) <br>- [ISO 8601](https://en.wikipedia.org/wiki/ISO_8601#Combined_date_and_time_representations) date string (e.g., `"2025-05-22T09:00:00Z"`)|
| `perturbed`          | If `True`, retrieves ensemble forecast members; if `False`, returns the deterministic forecast. |
| `horizon`            | Forecast lead time, provided as either:<br>– [datetime.timedelta](https://docs.python.org/3/library/datetime.html#timedelta-objects) object (e.g., `datetime.timedelta(hours=0)`) <br>– [ISO 8601](https://en.wikipedia.org/wiki/ISO_8601#Durations) duration string (e.g., `"P0DT0H"`)|

### Retrieving Data
We now send our request to the API and retrieve the resulting dataset using the [get_from_ogd()](https://meteoswiss.github.io/meteodata-lab/generated/meteodatalab.ogd_api.html#meteodatalab.ogd_api.get_from_ogd) function.
The response is returned as an **[xarray.DataArray](https://docs.xarray.dev/en/stable/generated/xarray.DataArray.html)**, which is efficient for handling multi-dimensional data.

> 💡 **Tip**: Use temporary caching with earthkit-data to skip repeated downloads — it's auto-cleaned after the session.
> *For more details, see the [earthkit-data caching docs](https://earthkit-data.readthedocs.io/en/latest/examples/cache.html)*.

> 💡 **Hint**: If you get an error message containing `HTTPError: 403 Client Error: Forbidden for url`, you may be trying to retrieve data older than 24h hours! Please adjust your requests.

In [ ]:
from earthkit.data import config
config.set("cache-policy", "temporary")

T = ogd_api.get_from_ogd(req)

The result of the `ogd_api.get_from_ogd()` call is an `xarray.DataArray` with the following structure:

**Dimensions**
- `eps` (ensemble members): 1 member (for deterministic data)
- `ref_time`: forecast init time (UTC)
- `lead_time`: time offset from `ref_time`
- `z`: **vertical model levels (80)**
- `cell`: spatial grid points (~280k for ICON-CH2-EPS)

**Coordinates**
- `lon` / `lat`: longitude & latitude per `cell`
- `valid_time`: `ref_time` + `lead_time`
        
**Attributes**
- `parameter`: variable info (e.g., name, units)
- `vcoord_type`: vertical coordinate type (e.g., `model_level`)
- `geography`: grid structure metadata

In [ ]:
T

> &#x1F525; **Expert users:** If you are interested in detailed metadata information, i.e. the GRIB encodings, you can retrieve it from the `metadata` attribute.
Please refer to the [earthkit-data metadata documentation](https://earthkit-data.readthedocs.io/en/latest/examples/metadata.html).

💡 To prepare for interpolation, let's inspect the initial vertical coordinate, the **model levels**:

In [ ]:
T.coords["z"].values

### Model levels

As shown in the output above, the parameter `T` includes temperature values on **80 model levels**. These model levels represent how the atmosphere is discretised from the Earth's surface up to the top of the model domain. In the numerical weather model ICON the 80 model levels correspond to the so-called **full levels** and are numbered top down. They are following the terrain and gradually change into levels of constant height as the distance from the surface increases (see the picture below). For more information about model levels, refer to the [model grid documentation](https://opendatadocs.meteoswiss.ch/e-forecast-data/e2-e3-numerical-weather-forecasting-model#vertical-grid).

<div style="text-align: center;">
  <img src="./images/VerticalLayers.png" width="50%">
</div>

## Interpolation from Model Levels to Target Altitude

Interpolating forecast data to a specific target altitude can be useful in certain applications. The transformation from model levels to a target altitude can be performed using meteodata-lab's [`interpolate_k2any()`](https://meteoswiss.github.io/meteodata-lab/generated/meteodatalab.operators.vertical_interpolation.html#meteodatalab.operators.vertical_interpolation.interpolate_k2any) function.

---
### How `interpolate_k2any()` Works

To interpolate a field from model levels to any vertical level the function `interpolate_k2any()`:

1. **Identifies all adjacent model levels** $k$ and $k-1$ such that:
   - $t_1 = \text{any\_level}[k-1]$ is just **above** target level $t$
   - $t_2 = \text{any\_level}[k]$ is just **below** target level $t$

2. **Determines the index $k$** depending on the selected `mode`:
    - "low_fold": Extracts the index $k$ of the highest matching target level along a coordinate
    - "high_fold": Extracts the index $k$ of the lowest matching target level along a coordinated
    
3. **Retrieves the field values** at the extracted level $k$ and $k-1$:
   - $f_1 = \text{field}[k-1]$
   - $f_2 = \text{field}[k]$

4. **Interpolates linearly**:

    $$
    r = \frac{t - t_1}{t_2 - t_1}, \quad f(t) = (1 - r) f_1 + r f_2
    $$

This logic is applied point-wise across the full 3D field.

### Implementation

In this example, we demonstrate how to interpolate temperature data on model levels to a specific target altitude.

The `interpolate_k2any()` function requires the following arguments:
- `field`: parameter to be interpolated
- `mode`: interpolation algorithm (`"low_fold"` or `"high_fold"`)
- `tc_field`: target field on model levels
- `tc`: target coordinates
- `h_field`: height field on model levels

In order to interpolate to a specific altitude the `tc_field` and `h_field` correspond to the same parameter: the height above sea level on **full model level (HFL)**. To derive the HFL data, we first retrieve the vertical grid parameters that provide the height above sea level at the **half levels (HHL)**. Next, we perform a destaggering operation on these half levels to compute the corresponding heights at the full levels.

In [ ]:
from meteodatalab import ogd_api
from meteodatalab import grib_decoder, data_source
from meteodatalab.operators.destagger import destagger
from meteodatalab.operators.vertical_interpolation import TargetCoordinates, TargetCoordinatesAttrs

# access vertical constant
url = ogd_api.get_collection_asset_url(
    collection_id="ch.meteoschweiz.ogd-forecasting-icon-ch2",
    asset_id="vertical_constants_icon-ch2-eps.grib2"
)
ds = grib_decoder.load(
    source=data_source.URLDataSource(urls=[url]),
    request={"param": "HHL"},
    geo_coords=lambda uuid: {}
)
HFL = destagger(ds["HHL"].squeeze(drop=True), "z")

Once retrieved the HFL data, continue with the interpolation.

> 💡 **Note**: The `mode` argument is not relevant for temperature data, as temperature decreases with height. However, when working with a target field that is non-monotonic, the choice of `mode`becomes important. Use "high_fold" to always select the highest matching target level along a coordinate, or "low_fold" to select the lowest.

In [ ]:
from meteodatalab.operators.vertical_interpolation import interpolate_k2any

# Define target coordinates
attrs = TargetCoordinatesAttrs("height_above_mean_sea_level", "height above the mean sea level", "m", "up")
target_coords = TargetCoordinates("heightAboveSea", [1000], attrs)

T_int = interpolate_k2any(
    field=T,
    mode="high_fold",
    tc_field=HFL,
    tc=target_coords,
    h_field=HFL
    )

### Target Altitude Inspection
After interpolation, the parameter now has updated vertical coordinates:
- `z`: 1000 masl (1 vertical level)
- `vcoord_type`: heightAboveSea (type of vertical coordinates)

In [ ]:
coords = T_int.coords["z"]
attrs = T_int.attrs["vcoord_type"]
print(f"\033[1mz \033[0m(vertical levels): {coords.values}")
print(f"\033[1mvcoord_type \033[0m(type of vertical coordinate): {attrs}")